In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import copy
import pprint
from benchmark_nerfstudio_paper import ablations_table_rows
from collections import defaultdict

# methods sorted in the same order we want in the paper tables
nerfacto_ablations_methods = [row[0] for row in ablations_table_rows]
print(nerfacto_ablations_methods)

['nerfacto', 'w/o-pose', 'w/o-app', 'w/o-pose-app', '1-prop-network', 'l2-contraction', 'shared-prop-network', 'random-background-color', 'no-contraction', 'synthetic-on-real']


In [3]:
def csv_filename_to_results(csv_filename, num_decimals=2, num_rows_to_consider=None):
    # code to read the csv file
    # and return a dictionary

    df = pd.read_csv(csv_filename)

    column_titles = list(df.columns.values)

    # iterate as long as the value is not None
    if num_rows_to_consider is None:
        rows = list(df.values)
    else:
        rows = list(df.values)[:num_rows_to_consider]
    valid_row = list(df.values[-1])
    for row in rows:
        for i in range(len(row)):
            value = row[i]
            if not pd.isnull(value):
                valid_row[i] = value
    last_row = valid_row
    # last_row will be the row that's valid

    results = defaultdict(dict)
    for column_title, last_row_value in zip(column_titles, last_row):
        if column_title == "Step" or "__MAX" in column_title or "__MIN" in column_title:
            continue
        name = column_title.split(" - ")[0]
        capture_name, method = name.split("_")
        metric = float(last_row_value)
        # format to have 2 decimal places
        if num_decimals == 2:
            metric = f"{metric:.2f}"
        elif num_decimals == 3:
            metric = f"{metric:.3f}"
        else:
            metric = f"{metric}"
        results[capture_name][method] = metric

    results = dict(results)
    return results

In [4]:
csv_filename_psnr = "./wandb/nerfacto_ablations_psnr.csv"
csv_filename_ssim = "./wandb/nerfacto_ablations_ssim.csv"
csv_filename_lpips = "./wandb/nerfacto_ablations_lpips.csv"
results_psnr = csv_filename_to_results(csv_filename_psnr, num_decimals=2)
results_ssim = csv_filename_to_results(csv_filename_ssim, num_decimals=3)
results_lpips = csv_filename_to_results(csv_filename_lpips, num_decimals=3)

In [5]:
def generate_row(method, capture_names):
    # remove spaces from method
    method_text = method.replace("-", " ")
    string = f"{method_text}"
    for idx, capture_name in enumerate(capture_names):
        psnr = results_psnr[capture_name][method]
        ssim = results_ssim[capture_name][method]
        lpips = results_lpips[capture_name][method]
        if idx == len(capture_names) - 1:
            string += " & \\multicolumn{1}{r|}{" + f"{psnr} / {ssim} / {lpips}" + "}"
        else:
            string += " & \\multicolumn{1}{r}{" + f"{psnr} / {ssim} / {lpips}" + "}"
    return string

In [6]:
capture_names_row1 = ["Egypt", "person", "kitchen", "plane", "dozer"]
capture_names_row2 = ["floating-tree", "aspen", "stump", "sculpture", "Giannini-Hall"]

print(capture_names_row1)
for method in nerfacto_ablations_methods[:-1]:
    print(generate_row(method, capture_names_row1) + " \\\ ")
print(generate_row(nerfacto_ablations_methods[-1], capture_names_row1) + " \\\ \hline")

print()

print(capture_names_row2)
for method in nerfacto_ablations_methods[:-1]:
    print(generate_row(method, capture_names_row2) + " \\\ ")
print(generate_row(nerfacto_ablations_methods[-1], capture_names_row2) + " \\\ \hline")

['Egypt', 'person', 'kitchen', 'plane', 'dozer']
nerfacto & \multicolumn{1}{r}{21.67 / 0.689 / 0.375} & \multicolumn{1}{r}{25.17 / 0.692 / 0.320} & \multicolumn{1}{r}{20.55 / 0.807 / 0.389} & \multicolumn{1}{r}{22.11 / 0.649 / 0.419} & \multicolumn{1}{r|}{20.20 / 0.743 / 0.391} \\ 
w/o pose & \multicolumn{1}{r}{21.56 / 0.689 / 0.374} & \multicolumn{1}{r}{25.52 / 0.716 / 0.323} & \multicolumn{1}{r}{20.47 / 0.798 / 0.402} & \multicolumn{1}{r}{21.83 / 0.645 / 0.421} & \multicolumn{1}{r|}{20.38 / 0.747 / 0.390} \\ 
w/o app & \multicolumn{1}{r}{22.38 / 0.689 / 0.396} & \multicolumn{1}{r}{27.74 / 0.744 / 0.316} & \multicolumn{1}{r}{23.80 / 0.819 / 0.397} & \multicolumn{1}{r}{22.44 / 0.641 / 0.440} & \multicolumn{1}{r|}{23.58 / 0.750 / 0.393} \\ 
w/o pose app & \multicolumn{1}{r}{22.31 / 0.692 / 0.399} & \multicolumn{1}{r}{27.76 / 0.749 / 0.328} & \multicolumn{1}{r}{23.39 / 0.808 / 0.409} & \multicolumn{1}{r}{22.22 / 0.642 / 0.443} & \multicolumn{1}{r|}{23.38 / 0.748 / 0.394} \\ 
1 prop netwo

In [7]:
def get_averages_from_results(results, num_decimals=2):
    averages = defaultdict(list)
    for capture_name, methods in results.items():
        for method, metric in methods.items():
            averages[method].append(float(metric))
    for method, metrics in averages.items():
        val = sum(metrics) / len(metrics)
        if num_decimals == 2:
            metric = f"{val:.2f}"
        elif num_decimals == 3:
            metric = f"{val:.3f}"
        else:
            metric = f"{val}"
        averages[method] = float(metric)
        
    return averages

results_average_psnr = get_averages_from_results(results_psnr, num_decimals=2)
results_average_ssim = get_averages_from_results(results_ssim, num_decimals=3)
results_average_lpips = get_averages_from_results(results_lpips, num_decimals=3)

for method in nerfacto_ablations_methods:
    row = f"{method} & {results_average_psnr[method]} & {results_average_ssim[method]} & {results_average_lpips[method]} \\\\"
    print(row)


nerfacto & 20.99 & 0.663 & 0.389 \\
w/o-pose & 20.93 & 0.659 & 0.393 \\
w/o-app & 22.65 & 0.672 & 0.406 \\
w/o-pose-app & 22.53 & 0.671 & 0.411 \\
1-prop-network & 21.07 & 0.669 & 0.396 \\
l2-contraction & 20.98 & 0.664 & 0.388 \\
shared-prop-network & 20.95 & 0.661 & 0.391 \\
random-background-color & 21.0 & 0.663 & 0.392 \\
no-contraction & 18.59 & 0.534 & 0.506 \\
synthetic-on-real & 20.09 & 0.542 & 0.509 \\
